# Project Imports

In [1]:
from project_utils import config_paths
config_paths('omri', 'my_env') # Change to your name and your environment folder!
from models import Unet

/home/omri/my_gpu/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from project_utils import config_paths
config_paths('omri', 'my_env') # Change to your name and your environment folder!
from project_utils import read_images, mean_iou, rle_encoding, prob_to_rles, predict_results, create_submission_file
from skimage.transform import resize
from skimage.morphology import label
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

In [3]:
main_tic = time.time()

# Constants

In [4]:
TRAIN_PATH = r'../../stage1_train/'
TEST_PATH = '../../stage1_test/'
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

# Read the data

In [5]:
X_train, Y_train, X_test, sizes_test  = read_images(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, TRAIN_PATH, TEST_PATH)

Getting and resizing train images and masks ... 


100%|██████████| 650/650 [01:21<00:00,  7.94it/s]

Getting and resizing test images ... 



100%|██████████| 65/65 [00:00<00:00, 96.76it/s] 

Done!


# Import/Build your model here:

In [42]:
model = Unet.unet(mean_iou, IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 128, 128, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 128, 128, 16) 448         lambda_4[0][0]                   
__________________________________________________________________________________________________
dropout_28 (Dropout)            (None, 128, 128, 16) 0           conv2d_58[0][0]                  
__________________________________________________________________________________________________
conv2d_59 

# Augment

In [29]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=False)

In [30]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [31]:
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=16),
                    steps_per_epoch=len(X_train) / 32, epochs=50,
                   validation_data=(X_val, Y_val))

Epoch 1/50
13/13 [==============================] - 6s 471ms/step - loss: 0.5305 - mean_iou: 0.3805 - val_loss: 0.4541 - val_mean_iou: 0.4245
Epoch 2/50
13/13 [==============================] - 3s 193ms/step - loss: 0.4711 - mean_iou: 0.4257 - val_loss: 0.3979 - val_mean_iou: 0.4281
Epoch 3/50
13/13 [==============================] - 3s 195ms/step - loss: 0.4022 - mean_iou: 0.4274 - val_loss: 0.3686 - val_mean_iou: 0.4286
Epoch 4/50
13/13 [==============================] - 3s 197ms/step - loss: 0.3709 - mean_iou: 0.4298 - val_loss: 0.3344 - val_mean_iou: 0.4320
Epoch 5/50
13/13 [==============================] - 3s 195ms/step - loss: 0.3663 - mean_iou: 0.4335 - val_loss: 0.3105 - val_mean_iou: 0.4346
Epoch 6/50
13/13 [==============================] - 3s 194ms/step - loss: 0.3461 - mean_iou: 0.4350 - val_loss: 0.2965 - val_mean_iou: 0.4361
Epoch 7/50
13/13 [==============================] - 3s 195ms/step - loss: 0.3475 - mean_iou: 0.4361 - val_loss: 0.2831 - val_mean_iou: 0.4367
Epoch 

# Train

In [43]:
tic = time.time()
model_name = 'model' 
model_name = model_name + '.h5'
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint(model_name, verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

toc = time.time() - tic
print('Training time...', toc/60)

Train on 374 samples, validate on 42 samples
Epoch 1/50
374/374 [==============================] - 9s 25ms/step - loss: 0.4217 - mean_iou: 0.4029 - val_loss: 0.2723 - val_mean_iou: 0.4289

Epoch 00001: val_loss improved from inf to 0.27230, saving model to model.h5
Epoch 2/50
374/374 [==============================] - 4s 11ms/step - loss: 0.2230 - mean_iou: 0.4643 - val_loss: 0.3530 - val_mean_iou: 0.5021

Epoch 00002: val_loss did not improve
Epoch 3/50
374/374 [==============================] - 4s 11ms/step - loss: 0.1573 - mean_iou: 0.5363 - val_loss: 0.2048 - val_mean_iou: 0.5699

Epoch 00003: val_loss improved from 0.27230 to 0.20475, saving model to model.h5
Epoch 4/50
374/374 [==============================] - 4s 11ms/step - loss: 0.1417 - mean_iou: 0.5930 - val_loss: 0.1238 - val_mean_iou: 0.6131

Epoch 00004: val_loss improved from 0.20475 to 0.12377, saving model to model.h5
Epoch 5/50
374/374 [==============================] - 4s 11ms/step - loss: 0.1241 - mean_iou: 0.6315 -

In [38]:
model.fit(X_train, Y_train, epochs=1, batch_size=16)

Epoch 1/1
416/416 [==============================] - 4s 10ms/step - loss: 0.0698 - mean_iou: 0.8432


# Predict

In [39]:
preds_test_upsampled = predict_results(model_name, X_train, X_test, sizes_test)

65/65 [==============================] - 0s 2ms/step


# Create a submission file

In [40]:
filename = 'predictions'
test_ids = os.listdir(TEST_PATH)
sub = create_submission_file(preds_test_upsampled, test_ids, filename)

In [41]:
sub.head()

,ImageId,EncodedPixels
0,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,46713 1 47231 4 47750 6 48268 7 48787 8 49306 ...
1,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,116259 2 116777 6 117296 7 117814 8 118333 8 1...
2,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,83565 3 84085 3 84604 5 85122 8 85641 10 86160...
3,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,24415 11 24932 16 25450 18 25967 21 26485 23 2...
4,0999dab07b11bc85fb8464fc36c947fbd8b5d6ec498173...,117327 8 117843 12 118361 14 118878 18 119396 ...


In [37]:
print('Total runtime...', (time.time() - main_tic)/60)

Total runtime... 49.06683441003164
